In [1]:
from numpy import array
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding

In [2]:

# source text
data = """Pichai Sundararajan (born June 10), better known as Sundar Pichai, is an American business executive.
He is the chief executive officer (CEO) of Alphabet Inc. and its subsidiary Google.Pichai began his career as a
materials engineer. Following a short stint at the management consulting firm McKinsey & Co., Pichai joined Google in 2004,
where he led the product management and innovation efforts for a suite of Google's client software products, including
Google Chrome and ChromeOS, as well as being largely responsible for Google Drive. In addition, he went on to oversee the
development of other applications such as Gmail and Google Maps. In 2010, Pichai also announced the open-sourcing of the new video codec VP8 by Google and introduced the new video format, WebM. The Chromebook was released in 2012. In 2013, Pichai added Android to the list of Google products that he oversaw.
Pichai was selected to become the next CEO of Google on August 10, 2015, after previously being appointed Product Chief by
CEO Larry Page. On October 24, 2015, he stepped into the new position at the completion of the formation of Alphabet Inc.,
the new holding company for the Google company family. He was appointed to the Alphabet Board of Directors in 2017.
Pichai was included in Time's annual list of the 100 most influential people in 2016[10] and 2020."""


In [3]:
# integer encode text
tokenizer = Tokenizer()
tokenizer.fit_on_texts([data])
encoded_data= tokenizer.texts_to_sequences([data])[0]
encoded_data

[4,
 33,
 34,
 35,
 12,
 36,
 37,
 8,
 38,
 4,
 18,
 39,
 40,
 41,
 19,
 6,
 18,
 1,
 20,
 19,
 42,
 13,
 2,
 14,
 21,
 7,
 43,
 44,
 3,
 4,
 45,
 46,
 47,
 8,
 15,
 48,
 49,
 50,
 15,
 51,
 52,
 22,
 1,
 23,
 53,
 54,
 55,
 56,
 4,
 57,
 3,
 5,
 58,
 59,
 6,
 60,
 1,
 24,
 23,
 7,
 61,
 62,
 16,
 15,
 63,
 2,
 64,
 65,
 66,
 25,
 67,
 3,
 68,
 7,
 69,
 8,
 70,
 8,
 26,
 71,
 72,
 16,
 3,
 73,
 5,
 74,
 6,
 75,
 17,
 9,
 76,
 1,
 77,
 2,
 78,
 79,
 80,
 8,
 81,
 7,
 3,
 82,
 5,
 83,
 4,
 84,
 85,
 1,
 86,
 87,
 2,
 1,
 10,
 27,
 88,
 89,
 28,
 3,
 7,
 90,
 1,
 10,
 27,
 91,
 92,
 1,
 93,
 11,
 94,
 5,
 95,
 5,
 96,
 4,
 97,
 98,
 9,
 1,
 29,
 2,
 3,
 25,
 99,
 6,
 100,
 4,
 11,
 101,
 9,
 102,
 1,
 103,
 13,
 2,
 3,
 17,
 104,
 12,
 30,
 105,
 106,
 26,
 31,
 24,
 20,
 28,
 13,
 107,
 108,
 17,
 109,
 110,
 30,
 6,
 111,
 112,
 1,
 10,
 113,
 22,
 1,
 114,
 2,
 1,
 115,
 2,
 14,
 21,
 1,
 10,
 116,
 32,
 16,
 1,
 3,
 32,
 117,
 6,
 11,
 31,
 9,
 1,
 14,
 118,
 2,
 119,
 5,
 120,
 4,
 1

In [4]:
# determine the vocabulary size
vocab_size = len(tokenizer.word_index) + 1  # 0 is reserved for padding so that's why we added 1

print('Vocabulary Size: %d' % vocab_size)



Vocabulary Size: 130


# Next, we need to create sequences of words to fit the model with one word as input and one word as output.


In [5]:
# create word -> word sequences
sequences = list()
for i in range(1, len(encoded_data)):
	sequence = encoded_data[i-1:i+1]
	sequences.append(sequence)
print('Total Sequences: %d' % len(sequences))
# split into X and y elements


Total Sequences: 225


# Running this piece shows that we have a total of 34 input-output pairs to train the network

In [6]:
sequences
#sequences[:5] # [input, output]

[[4, 33],
 [33, 34],
 [34, 35],
 [35, 12],
 [12, 36],
 [36, 37],
 [37, 8],
 [8, 38],
 [38, 4],
 [4, 18],
 [18, 39],
 [39, 40],
 [40, 41],
 [41, 19],
 [19, 6],
 [6, 18],
 [18, 1],
 [1, 20],
 [20, 19],
 [19, 42],
 [42, 13],
 [13, 2],
 [2, 14],
 [14, 21],
 [21, 7],
 [7, 43],
 [43, 44],
 [44, 3],
 [3, 4],
 [4, 45],
 [45, 46],
 [46, 47],
 [47, 8],
 [8, 15],
 [15, 48],
 [48, 49],
 [49, 50],
 [50, 15],
 [15, 51],
 [51, 52],
 [52, 22],
 [22, 1],
 [1, 23],
 [23, 53],
 [53, 54],
 [54, 55],
 [55, 56],
 [56, 4],
 [4, 57],
 [57, 3],
 [3, 5],
 [5, 58],
 [58, 59],
 [59, 6],
 [6, 60],
 [60, 1],
 [1, 24],
 [24, 23],
 [23, 7],
 [7, 61],
 [61, 62],
 [62, 16],
 [16, 15],
 [15, 63],
 [63, 2],
 [2, 64],
 [64, 65],
 [65, 66],
 [66, 25],
 [25, 67],
 [67, 3],
 [3, 68],
 [68, 7],
 [7, 69],
 [69, 8],
 [8, 70],
 [70, 8],
 [8, 26],
 [26, 71],
 [71, 72],
 [72, 16],
 [16, 3],
 [3, 73],
 [73, 5],
 [5, 74],
 [74, 6],
 [6, 75],
 [75, 17],
 [17, 9],
 [9, 76],
 [76, 1],
 [1, 77],
 [77, 2],
 [2, 78],
 [78, 79],
 [79, 80],

# We can then split the sequences into input (X) and output elements (y)

In [7]:
sequences = array(sequences)
X, y = sequences[:,0],sequences[:,1]



In [8]:
X[:5]

array([ 4, 33, 34, 35, 12])

In [9]:
y[:5]

array([33, 34, 35, 12, 36])

In [10]:
# one hot encode outputs
y = to_categorical(y, num_classes=vocab_size)
# define model
y[:5]

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,

In [11]:
model = Sequential()
model.add(Embedding(vocab_size, 10, input_length=1))
model.add(LSTM(50))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 1, 10)             1300      
                                                                 
 lstm (LSTM)                 (None, 50)                12200     
                                                                 
 dense (Dense)               (None, 130)               6630      
                                                                 
Total params: 20,130
Trainable params: 20,130
Non-trainable params: 0
_________________________________________________________________
None


In [12]:
# compile network
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])



In [13]:
# fit network
model.fit(X, y, epochs=100)


Epoch 1/100
8/8 [==============================] - 2s 3ms/step - loss: 4.8672 - accuracy: 0.0444 
Epoch 2/100
8/8 [==============================] - 0s 4ms/step - loss: 4.8633 - accuracy: 0.0800
Epoch 3/100
8/8 [==============================] - 0s 3ms/step - loss: 4.8600 - accuracy: 0.0800
Epoch 4/100
8/8 [==============================] - 0s 4ms/step - loss: 4.8566 - accuracy: 0.0756
Epoch 5/100
8/8 [==============================] - 0s 4ms/step - loss: 4.8530 - accuracy: 0.0756
Epoch 6/100
8/8 [==============================] - 0s 4ms/step - loss: 4.8491 - accuracy: 0.0756
Epoch 7/100
8/8 [==============================] - 0s 3ms/step - loss: 4.8446 - accuracy: 0.0756
Epoch 8/100
8/8 [==============================] - 0s 3ms/step - loss: 4.8401 - accuracy: 0.0756
Epoch 9/100
8/8 [==============================] - 0s 4ms/step - loss: 4.8349 - accuracy: 0.0756
Epoch 10/100
8/8 [==============================] - 0s 3ms/step - loss: 4.8288 - accuracy: 0.0756
Epoch 11/100
8/8 [==========

8/8 [==============================] - 0s 6ms/step - loss: 2.6875 - accuracy: 0.3600
Epoch 85/100
8/8 [==============================] - 0s 4ms/step - loss: 2.6623 - accuracy: 0.3600
Epoch 86/100
8/8 [==============================] - 0s 5ms/step - loss: 2.6379 - accuracy: 0.3689
Epoch 87/100
8/8 [==============================] - 0s 5ms/step - loss: 2.6132 - accuracy: 0.3822
Epoch 88/100
8/8 [==============================] - 0s 5ms/step - loss: 2.5894 - accuracy: 0.4000
Epoch 89/100
8/8 [==============================] - 0s 4ms/step - loss: 2.5659 - accuracy: 0.4000
Epoch 90/100
8/8 [==============================] - 0s 4ms/step - loss: 2.5430 - accuracy: 0.4044
Epoch 91/100
8/8 [==============================] - 0s 4ms/step - loss: 2.5200 - accuracy: 0.4089
Epoch 92/100
8/8 [==============================] - 0s 4ms/step - loss: 2.4961 - accuracy: 0.4133
Epoch 93/100
8/8 [==============================] - 0s 5ms/step - loss: 2.4729 - accuracy: 0.4178
Epoch 94/100
8/8 [===============

In [14]:
#enter_text = provide
n_pred = 3

#if in_text:
 #   result = provide


In [15]:
import numpy as np

def generate_seq(model, tokenizer, enter_text, n_pred):
    in_text = enter_text
    result = enter_text
    
    # generate a fixed number of words
    for _ in range(n_pred):
        # encode the text as integers
        encoded = tokenizer.texts_to_sequences([in_text])[0]
        
        # predict the probabilities of the next word
        yhat_probs = model.predict(np.array([encoded]))[0]
        
        # map predicted probability to word
        yhat_index = np.argmax(yhat_probs)
        out_word = None
        for word, index in tokenizer.word_index.items():
            if index == yhat_index:
                out_word = word
                break
        
        if out_word is None:
            break
        
        # append to input
        in_text = out_word
        result += ' ' + out_word
    
    return result


In [16]:
# evaluate
print(generate_seq(model, tokenizer, 'pichai', 6))

1/1 [==============================] - 0s 23ms/step
pichai was as being as being as


In [18]:
# evaluate
print(generate_seq(model, tokenizer, 'alphabet', 7))

1/1 [==============================] - 0s 29ms/step
alphabet inc the new video codec inc the
